In [1]:
from transformers import (
    TFGPT2LMHeadModel,
    GPT2Config,
    GPT2Tokenizer,
)
import tensorflow as tf

/home/ozymandias/llms/vteste/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-12 08:59:39.651808: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 08:59:39.854854: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-12 08:59:40.035730: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741780780.185475    7109 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741780780.228141    7109 cuda_blas.cc:1418]

In [2]:
model = TFGPT2LMHeadModel.from_pretrained("openai-community/gpt2")

W0000 00:00:1741780785.314723    7109 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124439808 
 er)                                                             
                                                                 
Total params: 124439808 (474.70 MB)
Trainable params: 124439808 (474.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")

In [ ]:
def padding(array):
    return tf.reshape(
        tf.convert_to_tensor(array),
        (-1, 32)
    )

In [ ]:
tkn1 = tokenizer("how are", return_tensors='tf')
tkn2 = tokenizer("What is your", return_tensors='tf')

In [ ]:
tkn1

In [ ]:
tkn2

In [ ]:
input_ids = tkn1['input_ids']
# input_ids = tf.convert_to_tensor([tkn1['input_ids'], tkn2['input_ids']])
# input_ids = padding([tokens['input_ids'][0][x] if x < tokens['input_ids'][0].shape[0] else 0 for x in range(32)])
input_ids

In [ ]:
attention_mask = tkn1['attention_mask']
# attention_mask = tf.convert_to_tensor([tkn1['attention_mask'], tkn2['attention_mask']])
# attention_mask = padding([1 if x < tokens['attention_mask'][0].shape[0] else 0 for x in range(32)])
attention_mask

In [ ]:
logits = model([input_ids]).logits[0:, -1]

In [ ]:
logits

In [ ]:
t = tf.math.argmax(logits[-1:,], axis=1)

In [ ]:
t

In [ ]:
tokenizer.decode(t)

## Apply Lora on GPT2

Consultar a classe [TFGPT2MainLayer](https://huggingface.co/transformers/v2.0.0/_modules/transformers/modeling_tf_gpt2.html)

In [ ]:
# class LoRALayer(tf.keras.layers.Layer):
#     def __init__(self, attn_layer: tf.keras.layers.Layer, rank: int):
#         super(LoRALayer, self).__init__()
#         self.old_layer = attn_layer
#         self.rank = rank

#         self.A_q = self.add_weight(
#             shape=(self.old_layer.n_head, 1, self.rank),
#             initializer='glorot_uniform',
#             trainable=True,
#             name="lora_qa")

#         self.B_q = self.add_weight(
#             shape=(1, self.rank, 64),
#             initializer='glorot_uniform',
#             trainable=True,
#             name="lora_qb")
        
#         self.A_k = self.add_weight(
#             shape=(self.old_layer.n_head, 1, self.rank),
#             initializer='glorot_uniform',
#             trainable=True,
#             name="lora_ka")

#         self.B_k = self.add_weight(
#             shape=(1, self.rank, 64),
#             initializer='glorot_uniform',
#             trainable=True,
#             name="lora_kb")

#         self.A_v = self.add_weight(
#             shape=(self.old_layer.n_head, 1, self.rank),
#             initializer='glorot_uniform',
#             trainable=True,
#             name="lora_va")

#         self.B_v = self.add_weight(
#             shape=(1, self.rank, 64),
#             initializer='glorot_uniform',
#             trainable=True,
#             name="lora_vb")

#         self.lora_alpha = 1.0

#     def call(self, inputs,
#         training=False,
#         layer_past=None,
#         attention_mask=None,
#         head_mask=None,
#         encoder_hidden_states=None,
#         encoder_attention_mask=None,
#         use_cache=None,
#         output_attentions=None,
#     ):
#         x = inputs
#         x = self.old_layer.c_attn(x)
#         query, key, value = tf.split(x, 3, axis=2)
#         query = self.old_layer.split_heads(query)
#         key = self.old_layer.split_heads(key)
#         value = self.old_layer.split_heads(value)
        
#         if layer_past is not None:
#             past_key, past_value = tf.unstack(layer_past, axis=1)
#             key = tf.concat([past_key, key], axis=-2)
#             value = tf.concat([past_value, value], axis=-2)
#         present = tf.stack([key, value], axis=1)
        
#         AB_q = tf.matmul(self.A_q, self.B_q)
        
#         AB_q = tf.reshape(AB_q, shape=(query.shape[0],) + AB_q.shape[0:])
#         AB_k = tf.matmul(self.A_k, self.B_k)
#         AB_k = tf.reshape(AB_k, shape=(key.shape[0],) + AB_k.shape[0:])
#         AB_v = tf.matmul(self.A_v, self.B_v)
#         AB_v = tf.reshape(AB_v, shape=(value.shape[0],) + AB_v.shape[0:])
        
#         query = query + 1 * AB_q
#         key = key + 1 * AB_k
#         value = value + 1 * AB_v
        
#         attn_outputs = self.old_layer._attn(q=query,k=key, v=value, attention_mask=attention_mask, head_mask=head_mask, training=training, output_attentions=output_attentions)
#         a = attn_outputs[0]

#         a = self.old_layer.merge_heads(a)
#         a = self.old_layer.c_proj(a)
#         a = self.old_layer.resid_dropout(a, training=training)

#         outputs = [a, present] + attn_outputs[1:]
#         return outputs
    

In [ ]:
RANK = 64

Identificando as camadas do modelo

In [ ]:
model.summary()

Precisamos aplicar o LoRA nas camadas do TFGPT2MainLayer bem como congelá-las

In [ ]:
print(f"NUM LAYERS: {model.transformer.num_hidden_layers}")

In [38]:
class LoraAttn(tf.keras.layers.Layer):
    def __init__(self, attn_layer: tf.keras.layers.Layer, rank: int, layer_id:int):
        super(LoraAttn, self).__init__(name="lora_layer")
        self.old_layer = attn_layer
        self.old_weights = self.old_layer.get_weights()
        self.rank = rank

        self.A = self.add_weight(
            shape=(768, self.rank),
            initializer='glorot_uniform',
            trainable=True,
            name=f"lora_qa_{layer_id}"
        )

        self.B = self.add_weight(
            shape=(self.rank, 2304),
            initializer='zeros',
            trainable=True,
            name=f"lora_qb_{layer_id}"
        )

        self.lora_alpha = 1.0
        
    def call(self, inputs):
        original = self.old_layer(inputs)

        output = tf.matmul(tf.matmul(inputs, self.A), self.B) * self.lora_alpha
        
        self.add_loss(0.0 * tf.reduce_sum(self.A))  
        self.add_loss(0.0 * tf.reduce_sum(self.B))

        return original + output



Como identificado precisamos alterar a variável 'h' que possuem a estrutura do modelo

O modelo possuem 12 camadas TFBlock

In [159]:
layer = model.transformer.h[0]

In [160]:
layer.mlp
# Para congelar
layer.mlp.trainable = False

In [43]:
layer.mlp.c_fc.weight.shape

TensorShape([768, 3072])

In [ ]:
dummy_input = tf.random.uniform(shape=(1, 10, 768))  # 1 exemplo, 10 tokens, 768 features

output = layer(
    dummy_input,
    layer_past=None,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    use_cache=None,
    output_attentions=None
)
print(output[0].shape)  # Esperado: (1, 10, 768)


(1, 10, 768)


In [ ]:
layer

In [ ]:
layer.mlp.c_proj.weight.shape

In [ ]:
out = layer.attn(tf.random.uniform(shape=(1, 10, 768)),  layer_past=None, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, use_cache=None, output_attentions=None)
out

In [ ]:
out[0].shape

In [ ]:
layer.attn.weights[-1].shape

In [ ]:
layer.attn.weights[1].shape

In [ ]:
layer.attn.c_proj.get_weights()[1].shape

In [ ]:
layer.attn

Cada TFBlock possue 2 Layer Normalization (ln_1 e ln_2) e uma multi layer perceptron (mlp) do tipo TFMLP e um attention (TFAttention)

In [315]:
model = TFGPT2LMHeadModel.from_pretrained("openai-community/gpt2")
# model.trainable = False

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [305]:
model.summary()

Model: "tfgpt2lm_head_model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  124439808 
 er)                                                             
                                                                 
Total params: 124439808 (474.70 MB)
Trainable params: 124439808 (474.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [306]:
trainable_params = model.trainable_variables
total_trainable_params = sum([tf.size(variable).numpy() for variable in trainable_params])

print(f"Total de parâmetros treináveis: {total_trainable_params}")

Total de parâmetros treináveis: 124439808


In [307]:
before_lora_params = model.count_params()
print(f"Total de parâmetros: {before_lora_params}")


Total de parâmetros: 124439808


In [ ]:
for i, tfblock in enumerate(model.transformer.h):
    original_attn = tfblock.attn.c_attn
    original_attn.trainable = False
    for var in original_attn.variables:
        var._trainable = False  # Congelamento manual
    setattr(tfblock.attn, "c_attn", LoraAttn(original_attn, rank=8, layer_id=i))


In [309]:
for i, tfblock in enumerate(model.transformer.h):
    print(f"Bloco {i} - {tfblock.attn.c_attn.name}, Treinável: {tfblock.attn.c_attn.trainable}")


Bloco 0 - lora_layer, Treinável: True
Bloco 1 - lora_layer, Treinável: True
Bloco 2 - lora_layer, Treinável: True
Bloco 3 - lora_layer, Treinável: True
Bloco 4 - lora_layer, Treinável: True
Bloco 5 - lora_layer, Treinável: True
Bloco 6 - lora_layer, Treinável: True
Bloco 7 - lora_layer, Treinável: True
Bloco 8 - lora_layer, Treinável: True
Bloco 9 - lora_layer, Treinável: True
Bloco 10 - lora_layer, Treinável: True
Bloco 11 - lora_layer, Treinável: True


In [ ]:
print("Lista de Variáveis Treináveis:")
for var in model.trainable_variables:
    print(f"{var.name}, Shape: {var.shape}")


🔍 Lista de Variáveis Treináveis:
tfgpt2lm_head_model_19/transformer/wte/embeddings:0, Shape: (50257, 768)
tfgpt2lm_head_model_19/transformer/wpe/embeddings:0, Shape: (1024, 768)
tfgpt2lm_head_model_19/transformer/h_._0/ln_1/gamma:0, Shape: (768,)
tfgpt2lm_head_model_19/transformer/h_._0/ln_1/beta:0, Shape: (768,)
tfgpt2lm_head_model_19/transformer/h_._0/attn/c_proj/weight:0, Shape: (768, 768)
tfgpt2lm_head_model_19/transformer/h_._0/attn/c_proj/bias:0, Shape: (1, 768)
lora_qa_0:0, Shape: (768, 8)
lora_qb_0:0, Shape: (8, 2304)
tfgpt2lm_head_model_19/transformer/h_._0/ln_2/gamma:0, Shape: (768,)
tfgpt2lm_head_model_19/transformer/h_._0/ln_2/beta:0, Shape: (768,)
tfgpt2lm_head_model_19/transformer/h_._0/mlp/c_fc/weight:0, Shape: (768, 3072)
tfgpt2lm_head_model_19/transformer/h_._0/mlp/c_fc/bias:0, Shape: (1, 3072)
tfgpt2lm_head_model_19/transformer/h_._0/mlp/c_proj/weight:0, Shape: (3072, 768)
tfgpt2lm_head_model_19/transformer/h_._0/mlp/c_proj/bias:0, Shape: (1, 768)
tfgpt2lm_head_model_

Precisamos recompilar o modelo porque se não tensorflow não considera ele no grafo computacional, bem como não podemos simplismente utilizar trainable = False e depois aplicar o lora, pois todos pesos são congelados recursivamente. A solução que encontrei foi congelar individualmente cada camada e garantir que o Lora não esteja congelado

In [ ]:
for i, tfblock in enumerate(model.transformer.h):
    original_attn = tfblock.attn.c_attn  # Pegamos a camada original
    original_attn.trainable = False  
    for var in original_attn.variables:
        var.assign(tf.stop_gradient(var))  # Remove gradiente
        var._trainable = False  # Congela os pesos antigos
    lora_layer = LoraAttn(original_attn, rank=8, layer_id=i)
    for var in lora_layer.variables:
        if "lora" in var.name:  # Somente os pesos LoRA podem ser treináveis
            var._trainable = True  
        else:
            var._trainable = False  # Congela qualquer outro peso herdado
    setattr(tfblock.attn, "c_attn", lora_layer)
model.transformer.wte.trainable = False  # Embeddings
model.transformer.wpe.trainable = False  # Embeddings
model.transformer.ln_f.trainable = False  # Última LayerNorm

for i, tfblock in enumerate(model.transformer.h):
    tfblock.ln_1.trainable = False  # LayerNorm 1
    tfblock.ln_2.trainable = False  # LayerNorm 2
    tfblock.attn.c_proj.trainable = False  # Projeção da atenção
    tfblock.mlp.c_fc.trainable = False  # Feedforward do MLP
    tfblock.mlp.c_proj.trainable = False  # Projeção final do MLP


lora_qa_0:0, Shape: (768, 8)
lora_qb_0:0, Shape: (8, 2304)
lora_qa_1:0, Shape: (768, 8)
lora_qb_1:0, Shape: (8, 2304)
lora_qa_2:0, Shape: (768, 8)
lora_qb_2:0, Shape: (8, 2304)
lora_qa_3:0, Shape: (768, 8)
lora_qb_3:0, Shape: (8, 2304)
lora_qa_4:0, Shape: (768, 8)
lora_qb_4:0, Shape: (8, 2304)
lora_qa_5:0, Shape: (768, 8)
lora_qb_5:0, Shape: (8, 2304)
lora_qa_6:0, Shape: (768, 8)
lora_qb_6:0, Shape: (8, 2304)
lora_qa_7:0, Shape: (768, 8)
lora_qb_7:0, Shape: (8, 2304)
lora_qa_8:0, Shape: (768, 8)
lora_qb_8:0, Shape: (8, 2304)
lora_qa_9:0, Shape: (768, 8)
lora_qb_9:0, Shape: (8, 2304)
lora_qa_10:0, Shape: (768, 8)
lora_qb_10:0, Shape: (8, 2304)
lora_qa_11:0, Shape: (768, 8)
lora_qb_11:0, Shape: (8, 2304)


In [ ]:
optimizer = model.optimizer if model.optimizer else tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer)
for var in model.trainable_variables:
    print(f"{var.name}, Shape: {var.shape}")


🔍 Lista Final de Variáveis Treináveis:
lora_qa_0:0, Shape: (768, 8)
lora_qb_0:0, Shape: (8, 2304)
lora_qa_1:0, Shape: (768, 8)
lora_qb_1:0, Shape: (8, 2304)
lora_qa_2:0, Shape: (768, 8)
lora_qb_2:0, Shape: (8, 2304)
lora_qa_3:0, Shape: (768, 8)
lora_qb_3:0, Shape: (8, 2304)
lora_qa_4:0, Shape: (768, 8)
lora_qb_4:0, Shape: (8, 2304)
lora_qa_5:0, Shape: (768, 8)
lora_qb_5:0, Shape: (8, 2304)
lora_qa_6:0, Shape: (768, 8)
lora_qb_6:0, Shape: (8, 2304)
lora_qa_7:0, Shape: (768, 8)
lora_qb_7:0, Shape: (8, 2304)
lora_qa_8:0, Shape: (768, 8)
lora_qb_8:0, Shape: (8, 2304)
lora_qa_9:0, Shape: (768, 8)
lora_qb_9:0, Shape: (8, 2304)
lora_qa_10:0, Shape: (768, 8)
lora_qb_10:0, Shape: (8, 2304)
lora_qa_11:0, Shape: (768, 8)
lora_qb_11:0, Shape: (8, 2304)


In [318]:
after_lora_params = model.count_params()
print(f"Total de parâmetros: {after_lora_params}")

Total de parâmetros: 124734720


In [319]:
print(f"Lora parameters add: {after_lora_params - before_lora_params}")


Lora parameters add: 294912


In [320]:
trainable_params = model.trainable_variables
total_trainable_params = sum([tf.size(variable).numpy() for variable in trainable_params])

print(f"Total de parâmetros treináveis: {total_trainable_params}")

Total de parâmetros treináveis: 294912


In [302]:
for layer in model.layers:
    print(layer.name, layer.trainable)


transformer True


In [1]:
print("Pesos treináveis:")
for var in model.trainable_variables:
    print(var.name)

total_params = model.count_params()
trainable_params = sum(tf.size(v).numpy() for v in model.trainable_variables)

print(f"TOTAL DE PARÂMETROS: {total_params}")
print(f"SÓ LO-RA TREINÁVEL: {trainable_params}")


Pesos treináveis:


NameError: name 'model' is not defined

In [155]:
input_ids = tokenizer("How are", return_tensors='tf')['input_ids']

In [18]:
import numpy as np
#1, 12, 3, 64
rank=58
matA = np.zeros(shape=(1, 10, rank))
matB = np.zeros(shape=(1, rank, 798))

In [19]:
np.matmul(matA, matB).shape

(1, 10, 798)

In [ ]:
a = np.zeros(shape=(1, 4, 768))
tf.split(a, 3, axis=2)

In [ ]:
a=[6.1173854,
5.804854,
6.236913,
7.7318535,
6.704432,
5.7643027,
6.005016,
5.6441536,
6.3495393,
5.589044,
6.232273,
6.3398476,
5.7454095,
7.5725613,
6.3798594,
6.8409615,
6.9971232,
5.8408737,
6.5225673,
6.8489685,
6.955326,
6.2124715,
6.583052,
6.4961452,
6.83617,
6.758698,
6.2365794,
5.210693,
6.3284087,
5.6087747,
6.641953,
7.1969275,
5.9185653,
5.8820357,
5.792418,
5.6274476,
7.6170025,
5.5744443,
6.745022,
6.573751,
6.9048324,
5.7228107,
5.795529,
6.276531,
5.8931737,
5.3276315,
5.7939715,
5.947024,
5.6362395,
6.774073,
5.738224,]

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(x=list(range(len(a))), y=a)